In [17]:
import edward as ed
import tensorflow as tf
from edward.models import Bernoulli, Normal
from scipy.misc import imsave
from tensorflow.contrib import slim
from tensorflow.examples.tutorials.mnist import input_data
from keras.layers import Dense

M = 1 # batch size during training
d = 2 # latent variable dimension

# Probability model (subgraph)
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))
h = Dense(256, activation='relu')(z)
x = Bernoulli(logits=Dense(28 * 28, activation=None)(h))

# Variational model (subgraph)
x_ph = tf.placeholder(tf.float32, [M, 28 * 28])
qh = Dense(256, activation='relu')(x_ph)
qz = Normal(loc=Dense(d, activation=None)(qh),
scale=Dense(d, activation='softplus')(qh))

# Bind p(x, z) and q(z | x) to the same TensorFlow placeholder for x.
mnist = input_data.read_data_sets("data/mnist", one_hot=True)
data = {x: x_ph}

inference = ed.KLqp({z: qz}, data)
optimizer = tf.train.RMSPropOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)

tf.initialize_all_variables().run()

n_epoch = 100
n_iter_per_epoch = 1000
for _ in range(n_epoch):
    for _ in range(n_iter_per_epoch):
        x_train, _ = mnist.train.next_batch(M)
        info_dict = inference.update(feed_dict={x_ph: x_train})

# Generate images.
imgs = x.value().eval()
for m in range(M):
    imsave("img/%d.png" % m, imgs[m].reshape(28, 28))


Extracting data/mnist\train-images-idx3-ubyte.gz
Extracting data/mnist\train-labels-idx1-ubyte.gz
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz


TypeError: Key-value pair in data does not have same dtype: <dtype: 'int32'>, <dtype: 'float32'>

In [16]:
x

<ed.RandomVariable 'Bernoulli_5/' shape=(100, 784) dtype=int32>

In [15]:
x_ph

<tf.Tensor 'Placeholder_1:0' shape=(100, 784) dtype=float32>